# NN MLP stacked model - 5 years

## Imports

In [0]:
from pyspark.sql.functions import col
from pyspark.sql import Window
import pyspark.sql.functions as F
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler
from pyspark.ml import Pipeline

from pyspark.ml.regression import LinearRegression, RandomForestRegressor
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import RegressionEvaluator, BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from xgboost.spark import SparkXGBRegressor

from mlflow.models import infer_signature


import random
import numpy as np
import pandas as pd

import mlflow
print(mlflow.__version__)

import os

spark.conf.set("spark.databricks.mlflow.trackMLlib.enabled", "true")

RANDOM_SEED = 0
# Define experiment name with proper Databricks path
EXPERIMENT_NAME = "/Shared/team_2_2/mlflow-nn-classifier"
# Create the experiment if it doesn't exist
try:
    experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
    if experiment is None:
        experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
        print(f"Created new experiment with ID: {experiment_id}")
    else:
        print(f"Using existing experiment: {experiment.name}")
    mlflow.set_experiment(EXPERIMENT_NAME)
except Exception as e:
    print(f"Error with experiment setup: {e}")
    # Fallback to default experiment in workspace
    mlflow.set_experiment(f"/Users/{dbutils.notebook.entry_point.getDbutils().notebook().getContext().userName().get()}/default")



## Helper Functions


In [0]:
def checkpoint_dataset(dataset, file_path):
    # Create base folder
    section = "2"
    number = "2"
    base_folder = f"dbfs:/student-groups/Group_{section}_{number}"
    dbutils.fs.mkdirs(base_folder)
    # Create subfolders if file_path contains directories
    full_path = f"{base_folder}/{file_path}.parquet"
    subfolder = "/".join(full_path.split("/")[:-1])
    dbutils.fs.mkdirs(subfolder)
    # Save dataset as a parquet file
    dataset.write.mode("overwrite").parquet(full_path)
    print(f"Checkpointed {file_path}")

## Datasets - custom join
- get checkpoint data
  - 5 year combined join, with feature engineering

# Feature Selection

In [0]:
baselines_columns = [
    "QUARTER",
    "MONTH",
    "YEAR",
    "DAY_OF_MONTH",
    "DAY_OF_WEEK",
    "OP_CARRIER",
    # "TAIL_NUM",
    "ORIGIN_AIRPORT_SEQ_ID",
    "DEST_AIRPORT_SEQ_ID",
    "CRS_ELAPSED_TIME",
    "DISTANCE",
    "DEP_DELAY_NEW",
    "DEP_DEL15",
    "utc_timestamp",
    "CRS_DEP_MINUTES",            # feature eng start
    "prev_flight_delay_in_minutes", 
    "prev_flight_delay",
    "origin_delays_4h",
    "delay_origin_7d",
    "delay_origin_carrier_7d",
    "delay_route_7d",
    "flight_count_24h",
    "LANDING_TIME_DIFF_MINUTES",
    "AVG_ARR_DELAY_ORIGIN",
    "AVG_TAXI_OUT_ORIGIN",        # feature eng end
    'HourlyDryBulbTemperature',     # weather start
    'HourlyDewPointTemperature',
    'HourlyRelativeHumidity',
    'HourlyAltimeterSetting',
    'HourlyVisibility',
    'HourlyStationPressure',
    'HourlyWetBulbTemperature',
    'HourlyPrecipitation',
    'HourlyCloudCoverage',
    'HourlyCloudElevation',
    'HourlyWindSpeed',               # weather end
    # 'page_rank',               # phase 3 new features start
    'out_degree',
    'in_degree',
    'weighted_out_degree',
    'weighted_in_degree',
    'N_RUNWAYS',
    'betweenness_unweighted',
    'closeness',
    'betweenness',
    'avg_origin_dep_delay',
    'avg_dest_arr_delay',
    'avg_daily_route_flights',
    'avg_route_delay',
    'avg_hourly_flights',
    "IS_HOLIDAY",
    "IS_HOLIDAY_WINDOW",
    "AIRPORT_HUB_CLASS",
    "RATING",
    "AIRLINE_CATEGORY",
    "ground_flights_last_hour",
    "arrivals_last_hour",
    "dow_sin",
    "dow_cos",
    "doy_sin",
    "doy_cos" # phase 3 new features end
]

In [0]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor

# Categorical encoding
carrier_indexer = StringIndexer(inputCol="OP_CARRIER", outputCol="carrier_idx", handleInvalid="keep")
origin_indexer = StringIndexer(inputCol="ORIGIN_AIRPORT_SEQ_ID", outputCol="origin_idx", handleInvalid="keep")
dest_indexer = StringIndexer(inputCol="DEST_AIRPORT_SEQ_ID", outputCol="dest_idx", handleInvalid="keep")
tail_num_indexer = StringIndexer(inputCol="TAIL_NUM", outputCol="tail_num_idx", handleInvalid="keep")
holiday_indexer = StringIndexer(inputCol="IS_HOLIDAY", outputCol="holiday_idx", handleInvalid="keep")
holiday_window_indexer = StringIndexer(inputCol="IS_HOLIDAY_WINDOW", outputCol="holiday_window_idx", handleInvalid="keep")
airport_hub_indexer = StringIndexer(inputCol="AIRPORT_HUB_CLASS", outputCol="airport_hub_idx", handleInvalid="keep")
airline_category_indexer = StringIndexer(inputCol="AIRLINE_CATEGORY", outputCol="airline_category_idx", handleInvalid="keep")

carrier_encoder = OneHotEncoder(inputCol="carrier_idx", outputCol="carrier_vec")
origin_encoder = OneHotEncoder(inputCol="origin_idx", outputCol="origin_vec")
dest_encoder = OneHotEncoder(inputCol="dest_idx", outputCol="dest_vec")
tail_num_encoder = OneHotEncoder(inputCol="tail_num_idx", outputCol="tail_num_vec")
holiday_encoder = OneHotEncoder(inputCol="holiday_idx", outputCol="holiday_vec")
holiday_window_encoder = OneHotEncoder(inputCol="holiday_window_idx", outputCol="holiday_window_vec")
airport_hub_encoder = OneHotEncoder(inputCol="airport_hub_idx", outputCol="airport_hub_vec")
airline_category_encoder = OneHotEncoder(inputCol="airline_category_idx", outputCol="airline_category_vec")


# # Inside ResFiLMMLP class:

# def __init__(self, ...):
#     # Create a list of embedding layers, one for each categorical column
#     # cat_dims = [Number of unique categories + 1 for Unknowns]
#     # emb_dims = [Vector size, e.g., 64]
#     self.embeddings = nn.ModuleList([
#         nn.Embedding(d, e) for d, e in zip(cat_dims, emb_dims)
#     ])

# def forward(self, x_cat, ...):
#     # Lookup the vector for each column's integer index
#     # x_cat[:, i] is the column of integers for category 'i'
#     emb = [emb_layer(x_cat[:, i]) for i, emb_layer in enumerate(self.embeddings)]
    
#     # Concatenate all vectors into one long feature vector
#     emb = torch.cat(emb, dim=-1)


In [0]:
# Assemble all features
assembler = VectorAssembler(
    inputCols=[
        "QUARTER",
        "MONTH", 
        "YEAR",
        "DAY_OF_MONTH",
        "DAY_OF_WEEK",
        "carrier_vec",
        "origin_vec",
        "dest_vec",
        "CRS_ELAPSED_TIME",
        "DISTANCE",
        # "tail_num_vec",
        "CRS_DEP_MINUTES",                 # feature eng start
        "prev_flight_delay_in_minutes",
        "prev_flight_delay",
        "origin_delays_4h",
        "delay_origin_7d",
        "delay_origin_carrier_7d",
        "delay_route_7d",
        "flight_count_24h",
        "LANDING_TIME_DIFF_MINUTES",
        "AVG_ARR_DELAY_ORIGIN",
        "AVG_TAXI_OUT_ORIGIN",              # feature eng end
        'HourlyDryBulbTemperature',         # weather start
        'HourlyDewPointTemperature',
        'HourlyRelativeHumidity',
        'HourlyAltimeterSetting',
        'HourlyVisibility',
        'HourlyStationPressure',
        'HourlyWetBulbTemperature',
        'HourlyPrecipitation',
        'HourlyCloudCoverage',
        'HourlyCloudElevation',
        'HourlyWindSpeed',                   # weather end
        # 'page_rank',               # phase 3 new features start
        'out_degree',
        'in_degree',
        'weighted_out_degree',
        'weighted_in_degree',
        'N_RUNWAYS',
        'betweenness_unweighted',
        'closeness',
        'betweenness',
        'avg_origin_dep_delay',
        'avg_dest_arr_delay',
        'avg_daily_route_flights',
        'avg_route_delay',
        'avg_hourly_flights',
        "holiday_vec",
        "holiday_window_vec",
        "airport_hub_vec",
        "RATING",
        "airline_category_vec",
        "ground_flights_last_hour",
        "arrivals_last_hour",
        "dow_sin",
        "dow_cos",
        "doy_sin",
        "doy_cos" # phase 3 new features end
    ],
    outputCol="raw_features"
)

# Training with Best Hyperparameters

In [0]:
# --- Evaluator (Use one metric for optimization) ---
rmse_evaluator = RegressionEvaluator(
    labelCol="DEP_DELAY_NEW",
    predictionCol="prediction",
    metricName="rmse" 
)

mae_evaluator = RegressionEvaluator(
        labelCol="DEP_DELAY_NEW",      
        predictionCol="prediction", 
        metricName="mae"    
)

precision_evaluator = MulticlassClassificationEvaluator(
    labelCol="DEP_DEL15",
    predictionCol="prediction",
    metricName="weightedPrecision"
)

recall_evaluator = MulticlassClassificationEvaluator(
    labelCol="DEP_DEL15",
    predictionCol="prediction",
    metricName="weightedRecall"
)

f1_evaluator = MulticlassClassificationEvaluator(
    labelCol="DEP_DEL15",
    predictionCol="prediction",
    metricName="f1"
)

f2_evaluator = MulticlassClassificationEvaluator(
    labelCol="DEP_DEL15",
    predictionCol="prediction",
    metricName="weightedFMeasure"
)
f2_evaluator.setBeta(2.0)

f2_evaluator_label = MulticlassClassificationEvaluator(
    labelCol="DEP_DEL15",
    predictionCol="prediction",
    metricName="fMeasureByLabel"
)
f2_evaluator_label.setMetricLabel(1.0).setBeta(2.0)

auc_evaluator = BinaryClassificationEvaluator(
    labelCol="DEP_DEL15", 
    rawPredictionCol="rawPrediction", 
    metricName="areaUnderROC"
)

acc_evaluator = MulticlassClassificationEvaluator(
    labelCol="DEP_DEL15", 
    predictionCol="prediction", 
    metricName="accuracy"
)

In [0]:
def read_specific_fold(path: str, fold_id: int, split_type: str):
    """
    Read a specific fold from partitioned parquet data.
    Falls back to filtering if direct partition read fails.
    """
    fold_path = f"{path}/fold_id={fold_id}/split_type={split_type}"
    
    try:
        # Try direct partition read
        return spark.read.parquet(fold_path)
    except:
        # Fallback: read all data and filter
        print(f"Direct read failed for fold {fold_id}, using filter method...")
        all_data = spark.read.parquet(path)
        return all_data.filter(
            (all_data.fold_id == fold_id) & 
            (all_data.split_type == split_type)
        )


# stacked approach (attempt 1)
- build XGBoost model and do hyperparamter tuning to find the best hyperparams
- generate the XGBoost regression delay field and output it using the held out data
- use the XGBoost delay field as the input for the NN/MLP model while doing hyperparameter tuning

In [0]:
from pyspark.ml.tuning import ParamGridBuilder
from xgboost.spark import SparkXGBRegressor
from pyspark.ml import Pipeline
from pyspark.sql import functions as F
from pyspark.storagelevel import StorageLevel
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.types import BooleanType
import itertools
import mlflow
import numpy as np
import pandas as pd
import shutil

# --- 1. Config ---
TRAIN_PARTITIONS = 6 
month_or_year = "5_year_custom_joined"
full_path = f"dbfs:/student-groups/Group_2_2/{month_or_year}/fe_graph_and_holiday_nnfeat/training_splits/"
train_path = full_path + "train.parquet"
val_path = full_path + "val.parquet"
test_path = full_path + "test.parquet"

# --- 2. Robust Safety Checks ---
# This REPLACES bad values instead of checking for them
@F.udf(returnType=VectorUDT())
def sanitize_vector(v):
    if v is None: return Vectors.dense([])
    
    # Convert to numpy for fast processing
    arr = v.toArray()
    
    # Replace NaN, Infinity, -Infinity with 0.0
    arr = np.nan_to_num(arr, nan=0.0, posinf=0.0, neginf=0.0)
    
    # Clip huge values to prevent float32 overflow in XGBoost
    arr = np.clip(arr, -1e30, 1e30)
    
    return Vectors.dense(arr)

# --- 3. Data Loading & Pipeline Definition ---
print(f"Loading and optimizing data from {month_or_year}...")

# Read in data
# full_df = spark.read.parquet(full_path).repartition(480).cache()
train_df = spark.read.parquet(train_path).repartition(480).cache()
val_df = spark.read.parquet(val_path).repartition(480).cache()
train_val_df = train_df.union(val_df).cache()
test_df = spark.read.parquet(test_path).repartition(480).cache()


# Define Final Model
best_depth = 6
best_estimators = 100
best_lr = 0.1   
final_xgb = SparkXGBRegressor(
    features_col="raw_features",
    label_col="DEP_DELAY_NEW",
    num_workers=6,
    tree_method="hist",
    max_depth=best_depth,
    n_estimators=best_estimators,
    learning_rate=best_lr,
    missing=0.0
)


# build pipeline stages list to use this specific assembler
xgb_pipeline_stages = [
    carrier_indexer, origin_indexer, dest_indexer, 
    tail_num_indexer, holiday_indexer, holiday_window_indexer, 
    airport_hub_indexer, airline_category_indexer,
    
    carrier_encoder, origin_encoder, dest_encoder, 
    tail_num_encoder, holiday_encoder, holiday_window_encoder, 
    airport_hub_encoder, airline_category_encoder,
    
    assembler,
    final_xgb
]

In [0]:
# with mlflow.start_run(run_name="FINAL_XGB_STACKED_5_YR") as parent_run:
global_pipeline = Pipeline(stages=xgb_pipeline_stages)

print("Fitting Global Feature Pipeline...")
final_model = global_pipeline.fit(train_val_df)

print("Transforming & Cleaning...")
train_val_predictions = final_model.transform(train_val_df)
test_predictions = final_model.transform(test_df)

In [0]:
# Calculate MAE
mae_train_val = mae_evaluator.evaluate(train_val_predictions)
mae_test = mae_evaluator.evaluate(test_predictions)
# Calculate RMSE
rmse_train_val = rmse_evaluator.evaluate(train_val_predictions)
rmse_test = rmse_evaluator.evaluate(test_predictions)

# Define DBFS paths
train_val_pred_path = full_path + "train_val_predictions"
test_pred_path = full_path + "test_predictions"
# Save as Parquet to DBFS
train_val_predictions.withColumnRenamed("prediction", "xgb_predicted_delay").write.mode("overwrite").parquet(train_val_pred_path)
test_predictions.withColumnRenamed("prediction", "xgb_predicted_delay").write.mode("overwrite").parquet(test_pred_path)

In [0]:
# Logging
with mlflow.start_run(run_name="FINAL_XGB_STACKED_5_YR"):

    # mlflow.log_artifacts(train_val_pred_path, "train_val_predictions")
    # mlflow.log_artifact(test_pred_path, "test_predictions")

    mlflow.log_metric("train_val_mae", mae_train_val)
    mlflow.log_metric("train_val_rmse", rmse_train_val)
    mlflow.log_metric("test_mae", mae_test)
    mlflow.log_metric("test_rmse", rmse_test)

    mlflow.log_param("max_depth", best_depth)
    mlflow.log_param("n_estimators", best_estimators)
    mlflow.log_param("learning_rate", best_lr)

    signature = infer_signature(train_val_df, train_val_predictions)

    mlflow.spark.log_model(
        final_model, 
        "final_stacked_xgb_model_5yr",
        input_example=train_val_df.limit(1).toPandas(),
        signature=signature,
        registered_model_name="final_stacked_xgb_model_5yr"
        )


In [0]:
display(dbutils.fs.ls("dbfs:/student-groups/Group_2_2/5_year_custom_joined/fe_graph_and_holiday_nnfeat/training_splits/"))

In [0]:
# display(dbutils.fs.ls("dbfs:/student-groups/Group_2_2/5_year_custom_joined/fe_graph_and_holiday_nnfeat/cv_splits/"))
display(dbutils.fs.ls("dbfs:/student-groups/Group_2_2/5_year_custom_joined/fe_graph_and_holiday_nnfeat/cv_splits2/"))
display(dbutils.fs.ls("dbfs:/student-groups/Group_2_2/5_year_custom_joined/fe_graph_and_holiday_nnfeat/cv_splits2/fold_id=1/"))

# stacked approach (attempt 2)

In [0]:
from xgboost.spark import SparkXGBRegressor
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
import mlflow
import time
import functools
from pyspark.sql import DataFrame
from pyspark.sql import functions as F

# --- 1. Config ---
TRAIN_PARTITIONS = 6 
month_or_year = "5_year_custom_joined"

# Base Paths
base_path = f"dbfs:/student-groups/Group_2_2/{month_or_year}/fe_graph_and_holiday_nnfeat/"
cv_splits_path = base_path + "cv_splits/"
training_splits_path = base_path + "training_splits/"

# Input Files (Final Model)
train_path = training_splits_path + "train.parquet"
val_path = training_splits_path + "val.parquet"
test_path = training_splits_path + "test.parquet"

# Output Paths
train_val_pred_path = training_splits_path + "train_val_predictions"
test_pred_path = training_splits_path + "test_predictions"

# --- 2. Feature Definitions (Ensuring availability) ---
xgb_stages_base = [
    carrier_indexer, origin_indexer, dest_indexer, 
    tail_num_indexer, holiday_indexer, holiday_window_indexer, 
    airport_hub_indexer, airline_category_indexer,
    carrier_encoder, origin_encoder, dest_encoder, 
    tail_num_encoder, holiday_encoder, holiday_window_encoder, 
    airport_hub_encoder, airline_category_encoder,
    assembler
]

# Hyperparameters
best_depth = 6
best_estimators = 100
best_lr = 0.1   

# --- 3. Execution ---
print("Starting Final Pipeline...")

oof_predictions = []
training_times = []
cv_train_mae = []
cv_train_rmse = []
cv_val_mae = []
cv_val_rmse = []

# Detect folds from file system or assume 1-10
folds = range(1, 11) 

with mlflow.start_run(run_name="FINAL_XGB_STACKED_OOF_5_YR") as run:
    
    # --- PART A: Generate Out-of-Fold Predictions (Using cv_splits2) ---
    print("Generating OOF Predictions from cv_splits2...")
    
    # Load all CV data once (lazy) to filter later
    cv_data = spark.read.parquet(cv_splits_path)
    
    for fold_id in folds:
        print(f"Processing Fold {fold_id}...")
        
        # 1. Filter Data for this Fold
        # Train on 'train', Predict on 'validation'
        train_fold = cv_data.filter((F.col("fold_id") == fold_id) & (F.col("split_type") == "train")).repartition(480).cache()
        val_fold = cv_data.filter((F.col("fold_id") == fold_id) & (F.col("split_type") == "validation")).repartition(480).cache()
        
        # 2. Define Model
        xgb = SparkXGBRegressor(
            features_col="raw_features",
            label_col="DEP_DELAY_NEW",
            num_workers=6,
            tree_method="hist",
            max_depth=best_depth,
            n_estimators=best_estimators,
            learning_rate=best_lr,
            missing=0.0
        )
        
        pipeline = Pipeline(stages=xgb_stages_base + [xgb])
        
        # 3. Train & Time
        start_time = time.time()
        model = pipeline.fit(train_fold)
        duration = time.time() - start_time
        training_times.append(duration)
        
        print(f"  Training took {duration:.2f} seconds")
        
        # 4. Predict & Evaluate (TRAIN)
        train_preds = model.transform(train_fold)
        t_mae = mae_evaluator.evaluate(train_preds)
        t_rmse = rmse_evaluator.evaluate(train_preds)
        
        cv_train_mae.append(t_mae)
        cv_train_rmse.append(t_rmse)
        
        # 5. Predict & Evaluate (VALIDATION)
        val_preds = model.transform(val_fold)
        v_mae = mae_evaluator.evaluate(val_preds)
        v_rmse = rmse_evaluator.evaluate(val_preds)
        
        cv_val_mae.append(v_mae)
        cv_val_rmse.append(v_rmse)
        
        print(f"  Fold {fold_id}: Time={duration:.2f}s")
        print(f"    Train: MAE={t_mae:.2f} | RMSE={t_rmse:.2f}")
        print(f"    Val:   MAE={v_mae:.2f} | RMSE={v_rmse:.2f}")
        
        # Log fold metrics
        mlflow.log_metric(f"fold_{fold_id}_train_mae", t_mae)
        mlflow.log_metric(f"fold_{fold_id}_val_mae", v_mae)
        mlflow.log_metric(f"fold_{fold_id}_train_rmse", t_rmse)
        mlflow.log_metric(f"fold_{fold_id}_val_rmse", v_rmse)
        
        # Keep predictions
        preds_clean = val_preds.withColumnRenamed("prediction", "xgb_predicted_delay")
        oof_predictions.append(preds_clean)

    # --- PART B: Train Final Model (For Test Set) ---
    print("\nTraining Final Model on ALL Train+Val Data...")
    
    # Load separate Train/Val files and union them
    train_df = spark.read.parquet(train_path)
    val_df = spark.read.parquet(val_path)
    train_val_df = train_df.union(val_df).repartition(480).cache()
    
    # Load Test
    test_df = spark.read.parquet(test_path).repartition(480).cache()
    
    final_xgb = SparkXGBRegressor(
        features_col="raw_features",
        label_col="DEP_DELAY_NEW",
        num_workers=6,
        tree_method="hist",
        max_depth=best_depth,
        n_estimators=best_estimators,
        learning_rate=best_lr,
        missing=0.0
    )
    final_pipeline = Pipeline(stages=xgb_stages_base + [final_xgb])
    
    # Train & Time
    start_time = time.time()
    final_model = final_pipeline.fit(train_val_df)
    final_duration = time.time() - start_time
    
    print(f"  Final training took {final_duration:.2f} seconds")
    
    # 1. Final Training Performance (Did we learn?)
    final_train_preds = final_model.transform(train_val_df)
    final_train_mae = mae_evaluator.evaluate(final_train_preds)
    final_train_rmse = rmse_evaluator.evaluate(final_train_preds)
    
    # 2. Test Set Performance (Did we generalize?)
    test_preds = final_model.transform(test_df)
    final_test_mae = mae_evaluator.evaluate(test_preds)
    final_test_rmse = rmse_evaluator.evaluate(test_preds)
    
    print(f"  Final Train: MAE={final_train_mae:.2f} | RMSE={final_train_rmse:.2f}")
    print(f"  Final Test:  MAE={final_test_mae:.2f}  | RMSE={final_test_rmse:.2f}")

    # --- PART C: Logging ---
    avg_train_time = sum(training_times) / len(training_times)
    
    mlflow.log_metric("avg_fold_training_time_sec", sum(training_times) / len(training_times))
    mlflow.log_metric("final_model_training_time_sec", final_duration)
    
    mlflow.log_metric("avg_cv_train_mae", np.mean(cv_train_mae))
    mlflow.log_metric("avg_cv_train_rmse", np.mean(cv_train_rmse))
    
    mlflow.log_metric("avg_cv_val_mae", np.mean(cv_val_mae))
    mlflow.log_metric("avg_cv_val_rmse", np.mean(cv_val_rmse))
    
    mlflow.log_metric("final_train_mae", final_train_mae)
    mlflow.log_metric("final_train_rmse", final_train_rmse)
    
    mlflow.log_metric("final_test_mae", final_test_mae)
    mlflow.log_metric("final_test_rmse", final_test_rmse)
    
    mlflow.log_param("max_depth", best_depth)
    mlflow.log_param("n_estimators", best_estimators)
    mlflow.log_param("learning_rate", best_lr)
    
    # Log Model
    signature = infer_signature(train_val_df, final_train_preds)

    mlflow.spark.log_model(
        final_model, 
        "final_stacked_xgb_model_5yr",
        input_example=train_val_df.limit(1).toPandas(),
        signature=signature,
        registered_model_name="final_stacked_xgb_model_5yr"
        )

    # --- PART D: Saving ---
    print("Saving Datasets...")
    
    # Union all OOF folds back together (These are your "Stacked" training features)
    full_train_val_with_preds = functools.reduce(DataFrame.union, oof_predictions)
    
    full_train_val_with_preds.write.mode("overwrite").parquet(train_val_pred_path)
    print(f"Saved OOF predictions to {train_val_pred_path}")
    
    test_preds_clean = test_preds.withColumnRenamed("prediction", "xgb_predicted_delay")
    test_preds_clean.write.mode("overwrite").parquet(test_pred_path)
    print(f"Saved Test predictions to {test_pred_path}")
    
    print("Success! Pipeline complete.")

In [0]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql import functions as F
from pyspark.storagelevel import StorageLevel
from pyspark.sql.types import BooleanType
import mlflow
from mlflow.models import infer_signature
import numpy as np
import time

# --- 1. Config ---
month_or_year = "5_year_custom_joined"
base_path = f"dbfs:/student-groups/Group_2_2/{month_or_year}/fe_graph_and_holiday_nnfeat/training_splits/"

# Input Paths (The outputs from your XGBoost step)
train_val_path = base_path + "train_val_predictions"
test_path = base_path + "test_predictions"

# Output Paths for Final Classification Results
final_train_output = base_path + "mlp_classification_train_val_predictions"
final_test_output = base_path + "mlp_classification_test_predictions"

# # --- 2. Safety UDF ---
# @F.udf(returnType=BooleanType())
# def vector_is_valid(v):
#     if v is None: return False
#     if np.any(np.isinf(v.values)): return False
#     if np.any(np.isnan(v.values)): return False
#     if np.max(np.abs(v.values)) > 1e30: return False
#     return True

SAFE_PARTITIONS = 2400

# --- 3. Data Loading ---
print("Loading Stacked Datasets...")

# Load Train/Val (Has OOF XGB predictions)
# We sample slightly just to infer schema/partitions quickly, but we use full data for training
train_df = spark.read.parquet(train_val_path).repartition(SAFE_PARTITIONS).cache()

# Load Test (Has standard XGB predictions)
test_df = spark.read.parquet(test_path).repartition(SAFE_PARTITIONS).cache()

print(f"Train Count: {train_df.count()}")
print(f"Test Count:  {test_df.count()}")

# --- 4. Pipeline Definition ---
# A. Define Indexers with NEW Output Names (_mlp)
# This avoids the "already exists" error and forces re-calculation
carrier_indexer = StringIndexer(inputCol="OP_CARRIER", outputCol="carrier_idx_mlp", handleInvalid="keep")
origin_indexer = StringIndexer(inputCol="ORIGIN_AIRPORT_SEQ_ID", outputCol="origin_idx_mlp", handleInvalid="keep")
dest_indexer = StringIndexer(inputCol="DEST_AIRPORT_SEQ_ID", outputCol="dest_idx_mlp", handleInvalid="keep")
tail_num_indexer = StringIndexer(inputCol="TAIL_NUM", outputCol="tail_num_idx_mlp", handleInvalid="keep")
holiday_indexer = StringIndexer(inputCol="IS_HOLIDAY", outputCol="holiday_idx_mlp", handleInvalid="keep")
holiday_window_indexer = StringIndexer(inputCol="IS_HOLIDAY_WINDOW", outputCol="holiday_window_idx_mlp", handleInvalid="keep")
airport_hub_indexer = StringIndexer(inputCol="AIRPORT_HUB_CLASS", outputCol="airport_hub_idx_mlp", handleInvalid="keep")
airline_category_indexer = StringIndexer(inputCol="AIRLINE_CATEGORY", outputCol="airline_category_idx_mlp", handleInvalid="keep")

# B. Define Encoders (Mapping _mlp indices to _mlp vectors)
carrier_encoder = OneHotEncoder(inputCol="carrier_idx_mlp", outputCol="carrier_vec_mlp")
origin_encoder = OneHotEncoder(inputCol="origin_idx_mlp", outputCol="origin_vec_mlp")
dest_encoder = OneHotEncoder(inputCol="dest_idx_mlp", outputCol="dest_vec_mlp")
tail_num_encoder = OneHotEncoder(inputCol="tail_num_idx_mlp", outputCol="tail_num_vec_mlp")
holiday_encoder = OneHotEncoder(inputCol="holiday_idx_mlp", outputCol="holiday_vec_mlp")
holiday_window_encoder = OneHotEncoder(inputCol="holiday_window_idx_mlp", outputCol="holiday_window_vec_mlp")
airport_hub_encoder = OneHotEncoder(inputCol="airport_hub_idx_mlp", outputCol="airport_hub_vec_mlp")
airline_category_encoder = OneHotEncoder(inputCol="airline_category_idx_mlp", outputCol="airline_category_vec_mlp")

# C. Assembler - Updates to use the NEW vectors (_mlp)
# INCLUDES 'xgb_predicted_delay'
assembler = VectorAssembler(
    inputCols=[
        "QUARTER", "MONTH", "YEAR", "DAY_OF_MONTH", "DAY_OF_WEEK",
        "carrier_vec_mlp", "origin_vec_mlp", "dest_vec_mlp",
        "CRS_ELAPSED_TIME", "DISTANCE", "CRS_DEP_MINUTES",
        "prev_flight_delay_in_minutes", "prev_flight_delay", "origin_delays_4h",
        "delay_origin_7d", "delay_origin_carrier_7d", "delay_route_7d",
        "flight_count_24h", "LANDING_TIME_DIFF_MINUTES", "AVG_ARR_DELAY_ORIGIN",
        "AVG_TAXI_OUT_ORIGIN", 'HourlyDryBulbTemperature', 'HourlyDewPointTemperature',
        'HourlyRelativeHumidity', 'HourlyAltimeterSetting', 'HourlyVisibility',
        'HourlyStationPressure', 'HourlyWetBulbTemperature', 'HourlyPrecipitation',
        'HourlyCloudCoverage', 'HourlyCloudElevation', 'HourlyWindSpeed',
        'out_degree', 'in_degree', 'weighted_out_degree', 'weighted_in_degree',
        'N_RUNWAYS', 'betweenness_unweighted', 'closeness', 'betweenness',
        'avg_origin_dep_delay', 'avg_dest_arr_delay', 'avg_daily_route_flights',
        'avg_route_delay', 'avg_hourly_flights',
        "holiday_vec_mlp", "holiday_window_vec_mlp", "airport_hub_vec_mlp",
        "RATING", "airline_category_vec_mlp",
        "ground_flights_last_hour", "arrivals_last_hour", "dow_sin", "dow_cos", "doy_sin", "doy_cos",
        "xgb_predicted_delay" # <--- The Stacking Feature
    ],
    outputCol="raw_mlp_features"
)

# Scaler (Critical for Neural Networks)
scaler = StandardScaler(inputCol="raw_mlp_features", outputCol="scaled_features", withStd=True, withMean=False)

# Preprocessing Pipeline
prep_pipeline = Pipeline(stages=[
    carrier_indexer, origin_indexer, dest_indexer, 
    tail_num_indexer, holiday_indexer, holiday_window_indexer, 
    airport_hub_indexer, airline_category_indexer,
    carrier_encoder, origin_encoder, dest_encoder, 
    tail_num_encoder, holiday_encoder, holiday_window_encoder, 
    airport_hub_encoder, airline_category_encoder,
    assembler, 
    scaler
])

print("Fitting Preprocessing Pipeline...")
prep_model = prep_pipeline.fit(train_df)

print("Transforming Data...")
full_train_vec = prep_model.transform(train_df).persist(StorageLevel.DISK_ONLY)
test_vec = prep_model.transform(test_df).persist(StorageLevel.DISK_ONLY)

# --- 5. FIX: Class Balancing (The Solution to F2=0) ---
print("Balancing Training Data...")
# Separate classes
positives = full_train_vec.filter(F.col("DEP_DEL15") == 1.0)
negatives = full_train_vec.filter(F.col("DEP_DEL15") == 0.0)

pos_count = positives.count()
neg_count = negatives.count()

print(f"  Found {pos_count} Positives and {neg_count} Negatives.")

# Undersample Negatives to match Positives
fraction = pos_count / neg_count
negatives_sampled = negatives.sample(withReplacement=False, fraction=fraction, seed=42)

# Combine and Repartition for training
train_balanced = positives.union(negatives_sampled).repartition(SAFE_PARTITIONS)
print(f"  Balanced Training Set: {train_balanced.count()} rows.")

# --- 6. MLP Definition & Training ---
input_dim = len(full_train_vec.first()["scaled_features"])
print(f"MLP Input Dimension: {input_dim}")

layers = [input_dim, 128, 64, 2]

mlp = MultilayerPerceptronClassifier(
    featuresCol="scaled_features",
    labelCol="DEP_DEL15",
    layers=layers,
    solver="gd",     # Gradient Descent for low memory usage
    blockSize=128,   
    stepSize=0.03,
    maxIter=100
)

# Evaluators
f2_label_evaluator = MulticlassClassificationEvaluator(labelCol="DEP_DEL15", metricName="fMeasureByLabel", metricLabel=1.0, beta=2.0)

print("Training Final MLP...")

with mlflow.start_run(run_name="FINAL_STACKED_MLP_5_YR") as run:
    mlflow.log_param("layers", str(layers))
    
    start_time = time.time()
    # TRAIN ON BALANCED DATA
    mlp_model = mlp.fit(train_balanced)
    training_time = time.time() - start_time
    
    mlflow.log_metric("training_duration_seconds", training_time)
    print(f"Training completed in {training_time:.2f} seconds")
    
    # --- 7. Evaluation ---
    print("Evaluating on Train (Balanced)...")
    train_preds = mlp_model.transform(train_balanced)
    t_f2 = f2_label_evaluator.evaluate(train_preds)
    print(f"  Train (Bal): Delay-F2={t_f2:.4f}")
    
    print("Evaluating on Test (Full/Imbalanced Holdout)...")
    test_preds = mlp_model.transform(test_vec)
    test_f2 = f2_label_evaluator.evaluate(test_preds)
    print(f"  Test:        Delay-F2={test_f2:.4f}")
    
    mlflow.log_metric("train_f2_delay", t_f2)
    mlflow.log_metric("test_f2_delay", test_f2)
    
    # --- 7b. FIX: Threshold Tuning (The Magic Step) ---
    print("\n--- Tuning Threshold on Full Training Data ---")

    # 1. Get predictions on the FULL (Imbalanced) training set
    # We need to tune on the real class distribution, not the balanced one.
    train_full_preds = mlp_model.transform(full_train_vec)

    # UDF to extract the probability of Class 1 (Delay)
    from pyspark.sql.types import DoubleType
    extract_prob = F.udf(lambda v: float(v[1]), DoubleType())

    # Add probability column
    train_with_prob = train_full_preds.withColumn("prob_delay", extract_prob(F.col("probability")))
    test_with_prob = test_preds.withColumn("prob_delay", extract_prob(F.col("probability")))

    # 2. Grid Search for Best Threshold
    best_threshold = 0.5
    best_f2 = 0.0
    thresholds = [0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]

    print("Searching for best threshold...")
    for t in thresholds:
        # Apply threshold dynamically
        preds_t = train_with_prob.withColumn("prediction", F.when(F.col("prob_delay") >= t, 1.0).otherwise(0.0))
        
        # Calculate F2
        f2 = f2_label_evaluator.evaluate(preds_t)
        print(f"  Threshold {t}: F2 = {f2:.4f}")
        
        if f2 > best_f2:
            best_f2 = f2
            best_threshold = t

    print(f"\nWINNER: Best Threshold = {best_threshold} (Train F2={best_f2:.4f})")
    mlflow.log_param("best_threshold", best_threshold)

    # 3. Apply Winner to TEST Set (Final Evaluation)
    print(f"Applying threshold {best_threshold} to Test Set...")
    final_test_preds = test_with_prob.withColumn("prediction", F.when(F.col("prob_delay") >= best_threshold, 1.0).otherwise(0.0))

    final_test_f2 = f2_label_evaluator.evaluate(final_test_preds)
    # final_test_prec = prec_evaluator.evaluate(final_test_preds)
    # final_test_rec = rec_evaluator.evaluate(final_test_preds)

    print(f"  FINAL TEST SCORE: Delay-F2 = {final_test_f2:.4f}")
    # print(f"  (Precision={final_test_prec:.4f}, Recall={final_test_rec:.4f})")

    mlflow.log_metric("final_test_f2_optimized", final_test_f2)

    # Update the dataframes to save so they include the optimized prediction
    train_preds = train_full_preds.withColumn("prediction", F.when(extract_prob(F.col("probability")) >= best_threshold, 1.0).otherwise(0.0))
    test_preds = final_test_preds

    # --- 8. FIX: Input Example Serialization ---
    # We must DROP vector columns from the example to avoid JSON errors
    # We pick just a few scalar columns to define the schema safely
    print("Logging Model...")
    
    # Identify vector columns to exclude from the example
    vector_cols = [c.name for c in train_df.schema if "VectorUDT" in str(c.dataType)]
    # Also exclude the generated _mlp vectors
    vector_cols += ["raw_mlp_features", "scaled_features"]
    
    # Create a clean example using only scalar columns
    # We take 1 row from the original DF and drop known vector columns
    example_pd = train_df.drop(*vector_cols).limit(1).toPandas()
    
    # Infer signature from the scalar inputs + prediction
    signature = infer_signature(example_pd, test_preds.limit(1).select("prediction").toPandas())

    mlflow.spark.log_model(
        mlp_model, 
        "final_stacked_mlp_model_5yr",
        input_example=example_pd, # Clean example
        signature=signature,
        registered_model_name="final_stacked_mlp_model_5yr"
    )
    
    # --- 9. Save Final Predictions ---
    print("Saving Final Predictions...")
    train_preds.write.mode("overwrite").parquet(final_train_output)
    test_preds.write.mode("overwrite").parquet(final_test_output)
    print("Success! Final Stacked Pipeline Complete.")

# Cleanup
train_df.unpersist()
test_df.unpersist()
full_train_vec.unpersist()
test_vec.unpersist()
train_balanced.unpersist()

# with mlflow.start_run(run_name="FINAL_STACKED_MLP_5_YR") as run:
#     mlflow.log_param("layers", str(layers))
    
#     # Train
#     start_time = time.time()
#     mlp_model = mlp.fit(train_vec)
#     training_time = time.time() - start_time
    
#     mlflow.log_metric("training_duration_seconds", training_time)
#     print(f"Training completed in {training_time:.2f} seconds")
    
#     # --- 6. Predictions & Evaluation ---
#     print("Evaluating on Train/Val (Self)...")
#     train_preds = mlp_model.transform(train_vec)
    
#     train_f2_delay = f2_label_evaluator.evaluate(train_preds)
    
#     print(f"  Delay-F2={train_f2_delay:.4f}")
    
#     print("Evaluating on Test (Holdout)...")
#     test_preds = mlp_model.transform(test_vec)
    
#     test_f2_delay = f2_label_evaluator.evaluate(test_preds)
    
#     print(f"  Delay-F2={test_f2_delay:.4f}")
    
#     # Log Metrics
#     mlflow.log_metric("train_f2_delay", train_f2_delay)

#     mlflow.log_metric("test_f2_delay", test_f2_delay)
    
#     # Log Model
#     mlflow.spark.log_model(mlp_model, "final_mlp_model")
#     signature = infer_signature(train_df, train_preds)

#     mlflow.spark.log_model(
#         mlp_model, 
#         "final_stacked_mlp_model_5yr",
#         input_example=train_df.limit(1).toPandas(),
#         signature=signature,
#         registered_model_name="final_stacked_mlp_model_5yr"
#         )
    
#     # --- 7. Save Final Predictions ---
#     print("Saving Final Predictions...")
    
#     # Join predictions back with original features if needed, or just save preds
#     # Here we save the simplified prediction output
#     train_preds.write.mode("overwrite").parquet(final_train_output)
#     test_preds.write.mode("overwrite").parquet(final_test_output)
    
#     print("Success! Final Stacked Pipeline Complete.")

# # Cleanup
# train_df.unpersist()
# test_df.unpersist()
# train_vec.unpersist()
# test_vec.unpersist()

In [0]:
# Create a custom thresholding UDF or logic
from pyspark.sql.types import DoubleType

# Function to extract probability of Class 1 and apply threshold
# Vector index 1 is the probability of Positive (Delay)
extract_prob = F.udf(lambda v: float(v[1]), DoubleType())

test_preds_prob = test_preds.withColumn("prob_delay", extract_prob(F.col("probability")))

# Check F2 at different thresholds
for t in [0.1, 0.2, 0.3, 0.4, 0.5]:
    # Manually classify based on threshold
    preds_thresholded = test_preds_prob.withColumn("prediction", F.when(F.col("prob_delay") >= t, 1.0).otherwise(0.0))
    
    f2 = f2_label_evaluator.evaluate(preds_thresholded)
    print(f"Threshold {t}: F2 Score = {f2:.4f}")